<a href="https://colab.research.google.com/github/mehmetbozdemir24/Tubitak_1505_Proje/blob/chunking/1505_project_Chunker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import sys
import pickle
import subprocess
import re

# Gerekli paketlerin kontrolü ve kurulumu
def install_packages():
    print("Gerekli kütüphaneler kuruluyor... Lütfen bekleyin.")
    packages = [
        "langchain",
        "langchain-community",
        "pypdf",
        "openpyxl",
        "pandas",
        "python-docx",
        "python-pptx",
        "unstructured",
        "networkx"
    ]
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)
        print("Kurulum tamamlandı.")
    except subprocess.CalledProcessError as e:
        print(f"Kurulum sırasında bir hata oluştu: {e}")
        # Optionally re-raise or handle more gracefully

try:
    import langchain
    import pandas as pd
    from docx import Document as DocxDocument
    from langchain_community.document_loaders import (
        PyPDFLoader,
        UnstructuredPowerPointLoader
    )
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_core.documents import Document
except ImportError:
    install_packages()
    # Re-attempt imports after installation
    import langchain
    import pandas as pd
    from docx import Document as DocxDocument
    from langchain_community.document_loaders import (
        PyPDFLoader,
        UnstructuredPowerPointLoader
    )
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_core.documents import Document


Gerekli kütüphaneler kuruluyor... Lütfen bekleyin.
Kurulum tamamlandı.


In [4]:
BASE_PATH = "/content/drive/MyDrive/1505 Chunking/Doküman"
TARGET_FOLDERS = ["pdf", "excel", "word", "powerpoint"]


In [5]:
def detect_permission_from_content(text_content):

    text_lower = text_content.lower()[:5000]

    # 1. Seviye: Admin
    admin_keywords = ["gizli", "confidential", "passwords", "şifreler", "yönetim kurulu", "admin only"]
    if any(k in text_lower for k in admin_keywords):
        return "admin"

    # 2. Seviye: Manager
    manager_keywords = ["maliyet", "bütçe", "budget", "finans", "salary", "maaş", "forecast"]
    if any(k in text_lower for k in manager_keywords):
        return "manager"

    # 3. Seviye: Editor
    editor_keywords = ["taslak", "draft", "düzenlenecek", "teknik şartname"]
    if any(k in text_lower for k in editor_keywords):
        return "editor"

    # Varsayılan: User
    return "user"



In [7]:
class DriveDocumentProcessor:
    def __init__(self, chunk_size=1000, chunk_overlap=100):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        self.processed_chunks = []

    def process_excel(self, file_path):

        try:
            xls = pd.read_excel(file_path, sheet_name=None)
            filename = os.path.basename(file_path)

            for sheet_name, df in xls.items():
                df.dropna(how='all', axis=0, inplace=True)
                df.dropna(how='all', axis=1, inplace=True)

                for col in df.columns:
                    if pd.api.types.is_datetime64_any_dtype(df[col]):
                        df[col] = df[col].dt.strftime('%Y-%m-%d')

                sheet_content_sample = df.to_string()
                permission = detect_permission_from_content(sheet_content_sample)

                columns = df.columns.tolist()

                for index, row in df.iterrows():
                    row_data = []
                    for col in columns:
                        val = row[col]
                        if pd.notna(val) and str(val).strip() != "":
                            row_data.append(f"{col}: {val}")

                    if not row_data: continue

                    row_text = (
                        f"KAYNAK: {filename} > {sheet_name}\n"
                        f"SATIR ID: {index+1}\n"
                        f"VERİLER:\n" + "\n".join(row_data)
                    )

                    metadata = {
                        "source": filename,
                        "sheet": sheet_name,
                        "row": index + 1,
                        "type": "excel_row",
                        "permission": permission,
                        "columns_present": list(columns)
                    }

                    self.processed_chunks.append(Document(page_content=row_text, metadata=metadata))

            print(f"  -> Excel işlendi: {filename} ({len(xls)} sayfa)")

        except Exception as e:
            print(f"  !! Excel hatası ({os.path.basename(file_path)}): {e}")

    def process_word_detailed(self, file_path):
        """
        Word dosyasını açar, paragrafları ayrı, tabloları ayrı işler.
        Tabloları Excel mantığıyla satır satır anlamlandırır.
        """
        try:
            doc = DocxDocument(file_path)
            filename = os.path.basename(file_path)

            full_text = "\n".join([p.text for p in doc.paragraphs])
            permission = detect_permission_from_content(full_text)

            text_content = []
            for para in doc.paragraphs:
                if para.text.strip():
                    text_content.append(para.text)

            if text_content:
                raw_text = "\n".join(text_content)
                text_chunks = self.text_splitter.create_documents([raw_text])
                for chunk in text_chunks:
                    chunk.metadata.update({
                        "source": filename,
                        "type": "word_text",
                        "permission": permission
                    })
                    self.processed_chunks.append(chunk)

            for i, table in enumerate(doc.tables):

                if not table.rows: continue

                headers = [cell.text.strip() for cell in table.rows[0].cells]

                if all(h == "" for h in headers):
                    headers = [f"Sütun_{k+1}" for k in range(len(headers))]

                for j, row in enumerate(table.rows[1:], start=1):
                    row_content = []
                    for k, cell in enumerate(row.cells):
                        if k < len(headers):
                            key = headers[k]
                            val = cell.text.strip()
                            if val:
                                row_content.append(f"{key}: {val}")

                    if row_content:

                        table_text = (
                            f"BELGE: {filename} > TABLO_{i+1}\n"
                            f"SATIR: {j}\n"
                            f"İÇERİK:\n" + "\n".join(row_content)
                        )

                        meta = {
                            "source": filename,
                            "type": "word_table",
                            "table_id": i+1,
                            "row_id": j,
                            "permission": permission
                        }
                        self.processed_chunks.append(Document(page_content=table_text, metadata=meta))

            print(f"  -> Word detaylı işlendi: {filename}")

        except Exception as e:
            print(f"  !! Word hatası ({filename}): {e}")

    def process_powerpoint_pdf(self, file_path, file_type):
        loader = None
        try:
            if file_type == "pdf":
                loader = PyPDFLoader(file_path)
            elif file_type == "powerpoint":
                loader = UnstructuredPowerPointLoader(file_path)

            if not loader: return

            raw_docs = loader.load()

            # İçerikten yetki analizi
            full_sample_text = " ".join([d.page_content for d in raw_docs[:5]])
            permission = detect_permission_from_content(full_sample_text)

            chunks = self.text_splitter.split_documents(raw_docs)
            filename = os.path.basename(file_path)

            for chunk in chunks:
                page = chunk.metadata.get("page", chunk.metadata.get("page_number", 1))
                chunk.metadata.update({
                    "source": filename,
                    "permission": permission,
                    "file_type": file_type
                })
                self.processed_chunks.append(chunk)

            print(f"  -> {file_type.upper()} işlendi: {len(chunks)} parça. (Yetki: {permission})")

        except Exception as e:
            print(f"  !! Hata ({file_type} - {os.path.basename(file_path)}): {e}")

    def save_data(self, output_path):
        with open(output_path, "wb") as f:
            pickle.dump(self.processed_chunks, f)
        print(f"\nToplam {len(self.processed_chunks)} chunk kaydedildi: {output_path}")


In [8]:
def main():
    processor = DriveDocumentProcessor()
    print(f"Tarama Başlıyor: {BASE_PATH}")

    if not os.path.exists(BASE_PATH):
        print(f"HATA: Belirtilen yol bulunamadı! -> {BASE_PATH}")

        return

    for folder in TARGET_FOLDERS:
        folder_path = os.path.join(BASE_PATH, folder)
        if not os.path.exists(folder_path):
            print(f"Uyarı: Klasör yok -> {folder}")
            continue

        print(f"\n[{folder.upper()}] Klasörü Taranıyor...")
        files = os.listdir(folder_path)

        for filename in files:
            file_path = os.path.join(folder_path, filename)

            # Gizli dosyaları atla
            if filename.startswith("."): continue

            # Dosya türüne göre işlem
            if folder == "excel" and (filename.endswith(".xlsx") or filename.endswith(".xls")):
                processor.process_excel(file_path)

            elif folder == "word" and (filename.endswith(".docx") or filename.endswith(".doc")):
                processor.process_word_detailed(file_path)

            elif folder == "pdf" and filename.endswith(".pdf"):
                processor.process_powerpoint_pdf(file_path, "pdf")

            elif folder == "powerpoint" and (filename.endswith(".pptx") or filename.endswith(".ppt")):
                processor.process_powerpoint_pdf(file_path, "powerpoint")

    output_file = "/content/drive/MyDrive/1505 Chunking/tum_dokumanlar_final.pkl"

    if not os.path.exists(os.path.dirname(output_file)):
        output_file = "tum_dokumanlar_final.pkl"

    processor.save_data(output_file)

if __name__ == "__main__":
    main()

Tarama Başlıyor: /content/drive/MyDrive/1505 Chunking/Doküman

[PDF] Klasörü Taranıyor...
  -> PDF işlendi: 67 parça. (Yetki: user)
  -> PDF işlendi: 53 parça. (Yetki: manager)
  -> PDF işlendi: 24 parça. (Yetki: user)
  -> PDF işlendi: 103 parça. (Yetki: user)
  -> PDF işlendi: 13 parça. (Yetki: user)
  -> PDF işlendi: 51 parça. (Yetki: user)
  -> PDF işlendi: 44 parça. (Yetki: user)
  -> PDF işlendi: 21 parça. (Yetki: admin)
  -> PDF işlendi: 27 parça. (Yetki: admin)
  -> PDF işlendi: 22 parça. (Yetki: user)
  -> PDF işlendi: 5 parça. (Yetki: admin)
  -> PDF işlendi: 7 parça. (Yetki: admin)
  -> PDF işlendi: 35 parça. (Yetki: user)
  -> PDF işlendi: 7 parça. (Yetki: admin)
  -> PDF işlendi: 14 parça. (Yetki: user)
  -> PDF işlendi: 5 parça. (Yetki: user)
  -> PDF işlendi: 15 parça. (Yetki: user)
  -> PDF işlendi: 7 parça. (Yetki: user)
  -> PDF işlendi: 5 parça. (Yetki: user)
  -> PDF işlendi: 3 parça. (Yetki: user)
  -> PDF işlendi: 3 parça. (Yetki: user)
  -> PDF işlendi: 4 parça. 

*view-chunker*

In [13]:
import os
import sys
import pickle


In [15]:
try:
    from langchain_core.documents import Document
except ImportError:
    import subprocess
    print("Gerekli kütüphane (langchain) kuruluyor...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "langchain", "langchain-community", "langchain-core"])
    from langchain_core.documents import Document

POSSIBLE_PATHS = [
    "/content/drive/MyDrive/1505 Chunking/tum_dokumanlar_final.pkl",
]

def find_file():
    for path in POSSIBLE_PATHS:
        if os.path.exists(path):
            return path
    return None

def preview_data(file_path):
    print(f" Yüklenen Dosya: {file_path}")

    try:
        with open(file_path, "rb") as f:
            chunks = pickle.load(f)
    except Exception as e:
        print(f"HATA: Dosya okunamadı. {e}")
        return

    total_count = len(chunks)
    print(f" Başarıyla Yüklendi! Toplam Chunk Sayısı: {total_count}")
    print("="*60)

    # Türlere göre grupla
    target_types = ["pdf", "excel", "word", "powerpoint"]
    samples = {t: [] for t in target_types}

    for chunk in chunks:
        ftype = chunk.metadata.get("file_type")

        if not ftype:
            source = chunk.metadata.get("source", "").lower()
            if source.endswith(".docx") or source.endswith(".doc"):
                ftype = "word"
            elif source.endswith(".xlsx") or source.endswith(".xls"):
                ftype = "excel"
            elif source.endswith(".pdf"):
                ftype = "pdf"
            elif source.endswith(".pptx") or source.endswith(".ppt"):
                ftype = "powerpoint"
            else:
                ftype = "unknown"

        if ftype in samples and len(samples[ftype]) < 3:
            samples[ftype].append(chunk)

    print(f"\n>>> Dosya Türü Bazlı Örnekler")

    for ftype in target_types:
        items = samples.get(ftype, [])
        print(f"\n--- TÜR: {ftype.upper()} ({len(items)} örnek gösteriliyor) ---")

        if not items:
            print("   (Bu dosya türüne ait veri bulunamadı)")
            continue

        for idx, item in enumerate(items):
            if ftype == 'excel':
                loc_label = "Satır"
                loc_val = item.metadata.get('row_number', '?')
                sheet_val = item.metadata.get('sheet', '')
                location_info = f"Sayfa: {sheet_val} | Satır: {loc_val}"
            elif ftype == 'word':
                if item.metadata.get('type') == 'word_table':
                    loc_label = "Tablo ID"
                    loc_val = item.metadata.get('table_id', '?')
                    location_info = f"Tablo: {loc_val} | Satır: {item.metadata.get('row_id', '?')}"
                else:
                    location_info = "Düz Metin"
            else:
                loc_label = "Sayfa"
                loc_val = item.metadata.get('page_number', item.metadata.get('page', '?'))
                location_info = f"Sayfa: {loc_val}"

            src = item.metadata.get('source')
            perm = item.metadata.get('permission', 'Yok')

            text = item.page_content.replace('\n', ' ')[:200]

            print(f"{idx+1}. Kaynak: {src}")
            print(f"   Konum : {location_info}")
            print(f"   Yetki : {perm}")
            print(f"   İçerik: {text}...")
            print("-" * 40)

found_path = find_file()

if found_path:
    preview_data(found_path)
else:
    print("HATA: Hiçbir .pkl dosyası bulunamadı!")
    print("Lütfen chunk işleminin başarıyla tamamlandığından ve dosyanın Drive'a kaydedildiğinden emin olun.")
    print("Aranan yollar:")
    for p in POSSIBLE_PATHS:
        print(f" - {p}")

 Yüklenen Dosya: /content/drive/MyDrive/1505 Chunking/tum_dokumanlar_final.pkl
 Başarıyla Yüklendi! Toplam Chunk Sayısı: 777

>>> Dosya Türü Bazlı Örnekler

--- TÜR: PDF (3 örnek gösteriliyor) ---
1. Kaynak: 207858.pdf
   Konum : Sayfa: 0
   Yetki : user
   İçerik: Senato Karar Eki    BURSA ULUDAĞ ÜNİVERSİTESİ İŞ SAĞLIĞI VE GÜVENLİĞİ YÖNERGESİ  BİRİNCİ BÖLÜM  Amaç, Kapsam, Dayanak ve Tanımlar  Amaç  MADDE 1- (1) Bu Yönerge’nin amacı, 6331 sayılı İş Sağlığı ve Gü...
----------------------------------------
2. Kaynak: 207858.pdf
   Konum : Sayfa: 0
   Yetki : user
   İçerik: Tanımlar  MADDE 4- (1) Bu Yönerge’de geçen,  a) Acil durum: İşyerinin tamamında veya bir kısmında meydana gelebilecek yangın,  patlama, tehlikeli kimyasal maddelerden kaynaklanan yayılım, doğal afet g...
----------------------------------------
3. Kaynak: 207858.pdf
   Konum : Sayfa: 0
   Yetki : user
   İçerik: d) Birim: Bir işyeri sosyal güvenlik sicil numarası ve/veya birim NACE kod numarasına  sahip olup İş Sağlı